# CNN分类
导入所需库

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

定义CNN模型

In [39]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=17):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 28 * 28, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


图片格式转化并加载数据

In [40]:


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


dataset = ImageFolder(root='E:\\code\\Jupyter\\final_repo\\pics', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
class_to_idx = dataset.class_to_idx
print(class_to_idx)

{'Analgesic': 0, 'Anti-inflammatory': 1, 'Antibacterial': 2, 'Antidepressant': 3, 'Antidiabetic': 4, 'Antifungal': 5, 'Antihistamine': 6, 'Antihypertensive': 7, 'Antioxidant': 8, 'Antiprotozoal': 9, 'Antipyretic': 10, 'Antispasmodic': 11, 'Antitumor': 12, 'Antiviral': 13, 'Diuretic': 14, 'Hypnotic': 15, 'Sedative': 16}


初始化模型

In [41]:

model = SimpleCNN(num_classes=17)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        train_acc = correct / total * 100
        train_loss = running_loss / len(train_loader)
        val_acc = evaluate(model, val_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%, Val Accuracy: {val_acc:.2f}%')
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_cnn_model2.pth')
    print(f"Training completed. Best validation accuracy: {best_val_acc:.2f}%")
# 评估函数
def evaluate(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total * 100


cuda


训练模型  
(之前训练过一轮，有图片证明，模型保存至CNN_Model.pth,不小心重启了一次)

In [42]:
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=30)

Epoch [1/30], Train Loss: 2.9968, Train Accuracy: 13.23%, Val Accuracy: 20.61%
Epoch [2/30], Train Loss: 2.4048, Train Accuracy: 21.45%, Val Accuracy: 28.33%


KeyboardInterrupt: 

不小心重启了内核，懒得重新训练，但还好之前的截图还在
![train](train.png)

测试测试集

In [6]:
# Load the best model and evaluate on validation set
model.load_state_dict(torch.load(r'E:\code\Jupyter\final_repo\CNN.pth'))
val_acc = evaluate(model, val_loader)
print(f"Final Validation Accuracy: {val_acc:.2f}%")

C:\Users\lenovo\AppData\Local\Temp\ipykernel_26916\1579089669.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_cnn_model.pth'))


Final Validation Accuracy: 43.23%


最后模型测试的函数及可视化放在了test2中